<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Silicon-neuron-and-synapse-models" data-toc-modified-id="Silicon-neuron-and-synapse-models-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Silicon neuron and synapse models</a></span></li><li><span><a href="#Single-neuron" data-toc-modified-id="Single-neuron-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Single neuron</a></span><ul class="toc-item"><li><span><a href="#Import-libraries" data-toc-modified-id="Import-libraries-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Import libraries</a></span></li><li><span><a href="#Creating-a-network-in-a-&quot;DYNAP-SE1-chip&quot;" data-toc-modified-id="Creating-a-network-in-a-&quot;DYNAP-SE1-chip&quot;-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Creating a network in a "DYNAP-SE1 chip"</a></span></li><li><span><a href="#Simulation-with-Poisson-input" data-toc-modified-id="Simulation-with-Poisson-input-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Simulation with Poisson input</a></span></li></ul></li></ul></div>

# Introduction to DYNAP-SE

By [Jingyue Zhao](https://www.ini.uzh.ch/fw1/modules/ini/ini.php/people/jzhao), [Elisa Donati](https://www.ini.uzh.ch/fw1/modules/ini/ini.php/people/elisa) and 
[Giacomo Indiveri](https://www.ini.uzh.ch/fw1/modules/ini/ini.php/people/giacomo) 

In this exercise we will learn how to set up a **neuron** and a **synapse**, connect them and form a **network**, and monitor the electrical signals at various stages in the chip simulator. 

## Silicon neuron and synapse models

<div class="alert alert-block alert-info">

In this tutorial, we will use the **Differential Pair Integrator (DPI)** neuron models of the neuromorphic chip, DYNAP-SE1, and we will simulate the 4 different types of synapses offered by DYNAP-SE1: **NMDA, AMPA, GABA_a and GABA_b**. 

Now let's look at the inside of the DYNAP-SE1 chip. Inside, there are four event-based, mixed-signal neuromorphic cores; each one has 256 DPI neurons. Each neuron is identical to the others by design; however, it may have slightly different parameters due to the noise and mismatch in analog circuits. 

***
**Brief introduction of DYNAP-SE1 neuronal dynamics**
***

The equations of the neuron and synapse models are given here, which you will learn in the lecture next week and we will not go into details here.

Differential Pair Integrator (DPI) circuit is used in DYNAP-SE1 to implement some fundamental dynamics of silicon synapses and Adaptive Exponential Integrate-and-Fire (AdExpIF) neurons.

Under some assumptions, the response dynamics of neuron block is: 

$$ \tau \frac{d}{d t} I_{m e m}+I_{m e m} \approx I_{i n} - I_{ahp} + f\left(I_{m e m}\right)$$ 

$$\tau_{ahp} \frac{d}{d t} I_{ahp} + I_{ahp}\approx I_{ahp} \delta\left(t_{spike}\right)
$$

Here, $I_{mem}$ is the sub-threshold current that represents the real neuron’s membrane potential variable, $I_{in}$ is the input current that enters the neuron, $I_{ahp}$ characterizes the spike adaptation effect, and $\tau$ is the time-constant of leakage current. $f\left(x\right)$ is an exponential function with positive exponent which characterizes the passive properties.

The simplified version of the response dynamics of the synaptic block is:

$$\tau \frac{d}{d t} I_{syn}(t) + I_{syn}(t)=I_{w}\delta\left(t_{pre}\right),$$

where $I_{syn}$ is the synaptic current, $I_w$ is the gain factor (weight) of the synapse, and $\tau$ is time-constant of current decay.
    
    
</div>    

As mentioned in previous session, our group has already implemented the neural and synaptic circuit models that DYNAP-SE1 uses on Brian2. Therefore, you don't have to worry about it's computational details.

If you want to have a look: Please feel free to wander around in the repository and dive into it's details later after you are done with the exercise

## Single neuron

### Import libraries

In [ ]:
from brian2 import *

**`DynapSE.py` wrapper class implements the DPI neuron, synaptic circuit models and monitors chip resources (e.g., number of available neurons per core, number of synapses between neurons, etc.)**

In [ ]:
from DynapSE import DynapSE

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# Display plots inside Jupyter cell
%matplotlib inline 
# Set the dots-per-inch (resolution) of the images
mpl.rcParams['figure.dpi'] = 90

The ordinary differential equations defining neuron and synapse models should be accessible by Brian2. Let's import the model equations and their parameters.

In [ ]:
from equations.dynapse_eq import *
from parameters.dynapse_param import *

# C++ code generation for faster spiking network simulation
set_device('cpp_standalone')
# Ignore Brian2 base warnings
BrianLogger.suppress_name('base')
# The clock of Brian2 simulation for numerically solve ODEs
defaultclock.dt = 20 * us

#### Some tips and a note on Hardware restrictions

<div class="alert alert-block alert-info">

**COREs**
***

In the simulation, you can allocate `num_n` neurons from a core `X` by using `get_neurons(num_n, 'Core_X')` method, where X range from 0 to 4, which returns a [SubGroup](https://brian2.readthedocs.io/en/stable/reference/brian2.groups.subgroup.Subgroup.html) of DPI neurons.

***
**CONNECTIVITY**
***

- An individual neuron can have a <b>fan-in of 64</b> and a <b>fan-out of 4k connections</b>. 
- The synapses exhibits one of 4 different behaviors: 
    - AMPA (excitatory), 
    - NMDA (excitatory), 
    - GABA_B (subtractive inhibitory), 
    - or GABA_A (shunting inhibitory). 
    

User can connect different neurons (which are obtained with `get_neurons()`) to each other using `add_connection(pre_population, post_population, synapse_type)` method. Synapse type can be either `AMPA`, `NMDA`, `GABA_B` and `GABA_A`.


***

**DPI neuronal parameters:**
***

- $Ith$: the gain factor of neuron.
- $Itau$: the membrane time-constant of leakage current.
- $refP$: refractory period.

***

**AMPA synaptic parameters:**
***

- I_tau_syn_ampa: corresponds to leakage current, i.e. how much current is constantly leaked away. It represents the time-constant of current decay.
- I_g_syn_ampa: the gain factor of the synapse.

These 2 parameters are implemented in the DYNAP-SE1 circuit. Besides, you still have `DPI_synapse.weight`. In Brian2, it's the weight of a single synapse, but in hardware reality, this corresponds to the number of physical connections you create between two silicon neurons. Thus, `DPI_synapse.weight` can only be integers.

<br/>
<br/>
If you need more details, please feel free to check initial parameter values (and units) in file "parameters/dynapse_param.py".
    
</div>    

### Creating a network in a "DYNAP-SE1 chip"

Do you remember `Network`?? Let's bring it back here. 
***

**Step 1:**
Create a **placeholder** (i.e, empty) `network` instance, we will pass it to the `DynapSE()` method to create a new `chip` instance.

***

In [ ]:
network = Network()
chip = DynapSE(network)

***

**Step 2:** Simulate a simple network, here are sequence of steps but you can do as you like as well.
***

**A:** Allocate a single neuron from the *any* core of DYNAP-SE1. Hint: use `get_neurons` from `DynapSE`. Syntax is provided for your convinence. `chip.get_neurons`

**B:** Send a single spike via the synapse. You can use `SpikeGeneratorGroup` from Brian2.

**C:** Define an AMPA synapse to connection a spike generator to this neuron. Hint: use `add_connection` from `DynapSE` and `connect` from Brian2.

**D:** Create a state monitor to monitor the output firing rate of the neuron. Use `StateMonitor` with **I_syn_ampa** as to_record variable.

**E:** Finally, add all the components into the `Network`.

**F:** Ofcourse, don't forget to simulate the network. Hint: use `network.run()`

**G:** One last thing, Plot the output current of AMPA synapse that enters to neuron. That's why we observed *I_syn_ampa*

Remember, this network is still running on your machine but the dynamics are similar to our neuromorphic chip 
***

<p style="color:red;font-size:20px;"> 
Attention!! Simulate for a very small time period, things can go extreamly slow. ~ 50ms is recommended
 </p>


In [ ]:
# ADD a Brian2 spikeGenerator and send 1 spike at 0th second
indices = [0]
times = [0]*second

spike_generator = #SpikeGeneratorGroup(1, indices, times)

# Allocate single neuron from Core 1
core_name = # which core, Core_#
neuron_num = # how many neuron
DPI_neuron  = chip.get_neurons(...count of neuron..., ..core name...)

# Connect the spike generator to the neuron using AMPA synapse 
DPI_synapse = chip.add_connection(...source.., ...target...., synapse_type=...synase_type....)

# In Brian2 creating a Synapses instance does not create synapses, it only specifies their dynamics.
# so remember to call connect function to really build the connection
chip.connect(DPI_synapse, True)

# here we set an initial weight of 200.
DPI_synapse.weight = # set your weight, try using 200 

In [ ]:
# monitors
mon_synapse_state = StateMonitor(...object to monitor..., ...variable to monitor..., record=[0])
mon_neuron_input  = SpikeMonitor(...object to monitor...,, name='mon_neuron_input')

# add all the instances into the network
network.add([...add elements here...])

In [ ]:
# Simulate for 50 ms
duration = 50 * ms
network.run(.....)

In [ ]:
# Plot change in current vs time, for a period of 0-10 ms
s = int(1*ms / defaultclock.dt)
tstart = 0 # ms 
tend   = 10 # ms
plt.figure(figsize=(4,2), dpi=150)
# todo
plt.plot(...read time...,...read value of current..)
plt.legend(['$I_{syn}$'])
plt.title('Synaptic Current')
plt.ylabel('Current (A)')
plt.xlabel('Time (ms)')
plt.grid(True)

In [ ]:
# Input Spikes, for verification 
times = int(duration/ms)

spike_placeholder = np.zeros(times)
spike_placeholder[np.array(mon_neuron_input.t/ms, dtype=int)]=1
plt.plot(spike_placeholder,'k|')
matplotlib.pyplot.yticks(range(1, 3))
plt.ylim(0.5, 1.5)
plt.title('Input Spikes')
plt.ylabel('Neuron ID')

What's the time constant of the AMPA synapse? Remember what you learned in the lecture? AMPA is a fast excitatory synapse.


<div class="alert alert-block alert-success">
    
<b>Answer the following as a Markdown cell</b>. 
- AMPA is excitatory synapse: True/False?
- What's the time constant of the AMPA synapse? 
- Is AMPA a slow or fast synapse.
</div>

***
**Add your answers here**
***



### Simulation with Poisson input
***

**Step 3:** Re-run your network for a Poisson input at 100 Hz, and check the output firing rate of the neuron.Simulate for 3 seconds, and check the firing rate of the neuron.

***
<b>A: Monitor the following neurons </b>
- I_syn_ampa
- Imem
- and record input and output spikes

<b>B: Plot Input spikes, membrane current, and output spikes to understand the neuron's response.</b>

<b>C: Feel free to adjust weight of your synapse
    
    
***
    

In [ ]:
# Add code here

Let's calculate the output firing rate of the neuron.

In [ ]:
print(mon_neuron_output.count/ duration)

Printing values for a few constant

In [ ]:
display(DPI_neuron.Ith)
display(DPI_neuron.Itau)
display(DPI_neuron.refP)
display(DPI_synapse.I_tau_syn_ampa)
display(DPI_synapse.I_g_syn_ampa)

<div class="alert alert-block alert-info">

***
**Bias Generator (BiasGen)**
***
**Usage :** It enables a user to adjust neuron and synapse parameters. For instance, membrane time constant.

**How it works? :** BiasGen is a configurable current generator that biases the gate voltage of specific transistors on the chip to generate the desired currents.

***
In the next exercise, we will tune parameter for AMPA synapse and neuron, to achieve the desired output firing rate.
***
    
</div>    